In [2]:
import pandas as pd
churn_csv = "WA_Fn-UseC_-Telco-Customer-Churn.csv"
df = pd.read_csv(churn_csv)

In [3]:
# remove customerID
df2 = df.drop(columns=["customerID"])

# replace the "no-internet-service" and "no-phone-service" with NaN
df2.replace(["No phone service", "No internet service", " "],[pd.NA, pd.NA, pd.NA], inplace=True)

# split columns like gender and internet service so that we can test the corelation, 
# and replace Yes wiht 1 and no with 0
df2 = pd.get_dummies(df2, columns=["gender", "InternetService", "Contract", "PaymentMethod"])

df2.replace(["Yes", "No"], [1, 0], inplace=True)

# normalize the data and convert str to float where needed
df2["TotalCharges"] = df2["TotalCharges"].dropna().astype(float)

normalized_df = df2

# removes each row with na value
normalized_df.dropna(inplace=True)

# normalizes the values of each column and replaces the values 
# of old column with values of the new column
for column in normalized_df:
    values = normalized_df.loc[:, column]
    max_value = max(values)
    if column == "InternetService_No":
        continue
    values = [value/max_value for value in values]
    normalized_df.loc[:, column] = values

/tmp/ipykernel_440263/377987156.py:29: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  normalized_df.loc[:, column] = values


In [4]:
from sklearn.model_selection import train_test_split
y = normalized_df['Churn']
x = normalized_df.drop(columns='Churn')

x_train, x_test, y_train, y_test = train_test_split(x, y)

In [7]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
layer_sizes = [[x,y,z] for x in range(100,300,100) 
               for y in range(0,200,100) for z in range(0,200,100)]

parms = {'hidden_layer_sizes':layer_sizes,
         'activation':['relu', 'logistic', 'tahn'],
         'alpha':[0.001, 0.01, 0.1, 1]}

grid = GridSearchCV(MLPClassifier(max_iter=3000),parms)
grid.fit(x,y)

/home/vincent/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
400 fits failed out of a total of 480.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
40 fits failed with the following error:
Traceback (most recent call last):
  File "/home/vincent/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/vincent/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py", line 747, in fit
    return self._fit(X, y, incremental=False)
  File "/home/vincent/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py", line 430, in 

AttributeError: 'MLPClassifier' object has no attribute 'score_samples'

In [13]:
parameters = grid.best_params_
scores = grid.best_score_

print(parameters)
print(scores)

{'activation': 'logistic', 'alpha': 0.001, 'hidden_layer_sizes': [200, 100, 100]}
0.7607610140859544
